In [0]:
%tensorflow_version 1.x

In [4]:
import tensorflow as tf
print(tf.__version__)

1.15.0


In [5]:
# Cloning BERT git repo
!git clone https://github.com/google-research/bert.git
!mv bert bert-master

Cloning into 'bert'...
remote: Enumerating objects: 336, done.
remote: Total 336 (delta 0), reused 0 (delta 0), pack-reused 336
Receiving objects: 100% (336/336), 290.62 KiB | 3.00 MiB/s, done.
Resolving deltas: 100% (184/184), done.


In [6]:
# Downloading BERT-Base, Cased Model
!wget https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip -O cased.zip

--2020-02-18 03:57:54--  https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.5.208, 2607:f8b0:4007:803::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.5.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 404261442 (386M) [application/zip]
Saving to: ‘cased.zip’

cased.zip           100%[===================>] 385.53M  59.5MB/s    in 6.5s    

2020-02-18 03:58:01 (59.5 MB/s) - ‘cased.zip’ saved [404261442/404261442]



In [7]:
# Unzipping the downloaded folder and placing it into root/model
!mkdir model/
!unzip cased.zip -d model/
!rm cased.zip
!cd model/cased_L-12_H-768_A-12 && mv * .. && cd .. && rm -r cased_L-12_H-768_A-12

# Moving the model folder into root/bert-master
!mv model bert-master

Archive:  cased.zip
   creating: model/cased_L-12_H-768_A-12/
  inflating: model/cased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: model/cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: model/cased_L-12_H-768_A-12/vocab.txt  
  inflating: model/cased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: model/cased_L-12_H-768_A-12/bert_config.json  


In [0]:
######################################################################################################
# We need to ensure we have the data in the following format before any of the code below can be run:
#   root/
#     data/
#       train.csv
#       test.csv
######################################################################################################

In [17]:
# # If needed:
# !pip3 install pandas, sklearn

import pandas as pd
from sklearn.model_selection import train_test_split

# Read data from csv file
df_train = pd.read_csv('dataset/imdb_train.csv')
df_test = pd.read_csv('dataset/imdb_test.csv')

# Creating the unique identifiers for the dataset
ids = []
for i in range(len(df_train)):
  ids.append(i)
pd_ids = pd.DataFrame(ids)
df_train.insert(0, "id", pd_ids)
# Converting sentiments from "positive" and "negative" to 1 and 0
df_train['sentiment'] = df_train['sentiment'].map({'positive': 1, 'negative': 0})

# Repeating the above processes for the test set
idx = []
for i in range(len(df_test)):
  idx.append(i)
pd_idx = pd.DataFrame(idx)
df_test.insert(0, "id", pd_idx)
df_test['sentiment'] = df_test['sentiment'].map({'positive': 1, 'negative': 0})

# # Create new dataframes in the format required by BERT for train, dev data
df_bert = pd.DataFrame({'guid': df_train['id'],
                        'label': df_train['sentiment'],
                        'alpha': ['a'] * df_train.shape[0],
                        'text': df_train['review']})

# Split into test, dev
df_bert_train, df_bert_dev = train_test_split(df_bert, test_size=0.01)

# Create new dataframe for test data
df_bert_test = pd.DataFrame({'guid': df_test['id'],
                             'text': df_test['review']})

# Output tsv file, no header for train and dev
!mkdir bert-master/dataset
df_bert_train.to_csv('bert-master/dataset/train.tsv', sep='\t', index=False, header=False)
df_bert_dev.to_csv('bert-master/dataset/dev.tsv', sep='\t', index=False, header=False)
df_bert_test.to_csv('bert-master/dataset/test.tsv', sep='\t', index=False, header=True)

print(df_bert_train.head())
print("-" * 100)
print(df_bert_test.head())

        guid  label alpha                                               text
18644  18644      0     a  About three minutes into this thing I started ...
19312  19312      0     a  I wanted to love this film so badly...I really...
33161  33161      1     a  The Ghost Train is a treat to those who apprec...
29031  29031      1     a  Set in a middle class neighborhood in the imag...
4901    4901      1     a  The late Director John Frankenheimer directed ...
----------------------------------------------------------------------------------------------------
   guid                                               text
0     0  One of the other reviewers has mentioned that ...
1     1  A wonderful little production. <br /><br />The...
2     2  I thought this was a wonderful way to spend ti...
3     3  Basically there's a family where a little boy ...
4     4  Petter Mattei's "Love in the Time of Money" is...


In [18]:
!nvidia-smi

Tue Feb 18 04:34:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [19]:
!cd bert-master && CUDA_VISIBLE_DEVICES=0 python3 run_classifier.py --task_name=cola --do_train=true --do_eval=true --data_dir=./dataset --vocab_file=./model/vocab.txt --bert_config_file=./model/bert_config.json --init_checkpoint=./model/bert_model.ckpt --max_seq_length=128 --train_batch_size=2 --learning_rate=2e-5 --num_train_epochs=3.0 --output_dir=./bert_output/ --do_lower_case=False --save_checkpoints_steps 1000

Streaming output truncated to the last 5000 lines.
I0218 08:09:04.616417 140554708506496 tpu_estimator.py:2307] global_step/sec: 4.70144
INFO:tensorflow:examples/sec: 9.40289
I0218 08:09:04.616635 140554708506496 tpu_estimator.py:2308] examples/sec: 9.40289
INFO:tensorflow:global_step/sec: 4.71057
I0218 08:09:04.828710 140554708506496 tpu_estimator.py:2307] global_step/sec: 4.71057
INFO:tensorflow:examples/sec: 9.42115
I0218 08:09:04.828954 140554708506496 tpu_estimator.py:2308] examples/sec: 9.42115
INFO:tensorflow:global_step/sec: 4.71037
I0218 08:09:05.041014 140554708506496 tpu_estimator.py:2307] global_step/sec: 4.71037
INFO:tensorflow:examples/sec: 9.42073
I0218 08:09:05.041203 140554708506496 tpu_estimator.py:2308] examples/sec: 9.42073
INFO:tensorflow:global_step/sec: 4.68511
I0218 08:09:05.254807 140554708506496 tpu_estimator.py:2307] global_step/sec: 4.68511
INFO:tensorflow:examples/sec: 9.37021
I0218 08:09:05.255046 140554708506496 tpu_estimator.py:2308] examples/sec: 9.3702